# 1. Định nghĩa cây B (B-Tree)

Cây nhị phân cân bằng thì tại mỗi nút của cây thì số nút cây con trái và số nút cây con phải hơn kém nhau nhiều nhất 1. Độ phức tạp của thuật toán là `O(log2n)`.

Tuy nhiên cây nhị phân cân bằng vẫn còn nhược điểm đó là **mỗi nút chỉ chứa một khóa** và khi thêm nhiều khóa vào sẽ **làm tăng chiều cao của cây** dẫn đến thuật toán chạy vòng lặp càng lâu hơn.

Để khắc phục hạn chế này người ta **đề xuất ra cây B.**

Một trang của cây B có nhiều trang con, một cây nhiều nhánh, khi số lượng khóa key tăng sẽ hạn chế việc tăng chiều cao của cây.

**Một số đặc điểm của cây B:**
+ Mỗi trang có tối đa 2xn khóa
+ Mỗi trang có ít nhất n khóa, ngoại trừ khóa gốc là có 1 khóa
+ Mỗi trang hoặc là trang lá có m + 1 khóa của trang này
+ Tất cả trang lá phải có cùng mức


<img align="left" width="500" height="500" src="images/B_Tree.png">

# 2. Cấu trúc dữ liệu
## 2.1. Cấu trúc dữ liệu của phần tử khóa
+ Key: Chứa khóa của phần tử
+ Next: Con trỏ trang con phải

## 2.2. Cấu trúc dữ liệu của trang
+ P0: Con trỏ trang đầu tiên
+ KeyItem: mảng chứa phần tử khóa của trang
+ KeyNum: Số khóa hiện tại của trang

## 2.3. Cấu trúc dữ liệu của cây
+ Root: Con trỏ trang gốc của cây B

# 3. Các thao tác với cây B
+ Khởi tạo cây 
+ Tìm kiếm khóa trong cây
+ Thêm khóa vào cây
+ Loại bỏ khóa trong cây
+ Duyệt cây

In [1]:
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.child = []
class BTree:
    def __init__(self, t):
        self.root = BTreeNode(True)
        self.t = t
        
    def insert(self, k):
        root = self.root
        if len(root.keys) == (2 * self.t) - 1:
            temp = BTreeNode()
            self.root = temp
            temp.child.insert(0, root)
            self.split_child(temp, 0)
            self.insert_non_full(temp, k)
        else:
            self.insert_non_full(root, k)

    # Chèn nonfull
    def insert_non_full(self, x, k):
        i = len(x.keys) - 1
        if x.leaf:
            x.keys.append((None, None))
            while i >= 0 and k[0] < x.keys[i][0]:
                x.keys[i + 1] = x.keys[i]
                i -= 1
            x.keys[i + 1] = k
        else:
            while i >= 0 and k[0] < x.keys[i][0]:
                i -= 1
            i += 1
            if len(x.child[i].keys) == (2 * self.t) - 1:
                self.split_child(x, i)
                if k[0] > x.keys[i][0]:
                    i += 1
            self.insert_non_full(x.child[i], k)

    # Cắt nút con
    def split_child(self, x, i):
        t = self.t
        y = x.child[i]
        z = BTreeNode(y.leaf)
        x.child.insert(i + 1, z)
        x.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t: (2 * t) - 1]
        y.keys = y.keys[0: t - 1]
        if not y.leaf:
            z.child = y.child[t: 2 * t]
            y.child = y.child[0: t - 1]

    def print_tree(self, x, l=0):
        print("Level ", l, " ", len(x.keys), end=":")
        for i in x.keys:
              print(i, end=" ")
        print()
        l += 1
        if len(x.child) > 0:
            for i in x.child:
                self.print_tree(i, l)

  # Tìm kiếm khóa trong cây
    def search_key(self, k, x=None):
        if x is not None:
            i = 0
            while i < len(x.keys) and k > x.keys[i][0]:
                i += 1
            if i < len(x.keys) and k == x.keys[i][0]:
                return (x, i)
            elif x.leaf:
                return None
            else:
                return self.search_key(k, x.child[i])

        else:
            return self.search_key(k, self.root)


def main():
    B = BTree(3)

    for i in range(10):
        B.insert((i, 2 * i))

    B.print_tree(B.root)

    if B.search_key(8) is not None:
        print("\nFound")
    else:
        print("\nNot Found")
if __name__ == '__main__':
    main()


Level  0   2:(2, 4) (5, 10) 
Level  1   2:(0, 0) (1, 2) 
Level  1   2:(3, 6) (4, 8) 
Level  1   4:(6, 12) (7, 14) (8, 16) (9, 18) 

Found
